# Utilizando o banco de dados raspado do site: https://genealogy.math.ndsu.nodak.edu

O banco de dados é composto por três tabelas do SQLite, organizando as informações da seguinte forma:

- researchers: Armazena o ID e o nome de cada pesquisador.
- academic_titles: Registra os detalhes de cada título acadêmico, vinculando-os a um pesquisador por meio do seu ID.
- advisors_academic_titles: Mapeia a relação entre orientadores e títulos acadêmicos, que, por sua vez, estão associados a pesquisadores.

## Bibliotecas:

In [ ]:
#!pip install faiss-cpu
#!pip install pyoperon
#!pip install gplearn

In [ ]:
import sqlite3
import json
#import pickle   # Armazenar e carregar modelo treinado
#import faiss
import os

import pandas as pd
import numpy as np

from tqdm import tqdm
from dotenv import load_dotenv
from collections import defaultdict
from openai import OpenAI, ChatCompletion
from pyoperon.sklearn import SymbolicRegressor
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import LabelEncoder#, StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
#from getpass import getpass

# Carregar variáveis de ambiente do arquivo .env
load_dotenv()

# Caminho onde o banco está/será salvo
db_sqlite = "./data/mgp.sqlite"

## Objetos:

In [ ]:
# Objeto pesquisador
class researcher:
  def __init__(self, researcher_id, name, academic_titles, descendants_id):
    self.researcher_id = researcher_id
    self.name = name
    self.academic_titles = academic_titles
    self.descendants_id = descendants_id

  def __str__(self):
    return f'{self.researcher_id} - {self.name} - {self.academic_titles} - {self.descendants_id}'

  def to_list(self):
    academic_titles_list = [academic_title.to_list() for academic_title in self.academic_titles]
    return [self.researcher_id, self.name, academic_titles_list, self.descendants_id]

  def save_to_db(self, db_sqlite):
    with sqlite3.connect(db_sqlite) as conn:
      cursor = conn.cursor()
      conn = sqlite3.connect(db_sqlite)
      cursor = conn.cursor()

      # Cria as tabelas se não existir
      cursor.execute('''
        CREATE TABLE IF NOT EXISTS researchers (
          researcher_id INTEGER PRIMARY KEY,
          name TEXT NOT NULL
        )
      ''')

      cursor.execute('''
        CREATE TABLE IF NOT EXISTS academic_titles (
          academic_title_id INTEGER PRIMARY KEY,
          researcher_id INTEGER NOT NULL,
          title TEXT NOT NULL,
          dissertation TEXT,
          institution TEXT,
          year INTEGER,
          country TEXT,
          FOREIGN KEY (researcher_id) REFERENCES researchers (researcher_id) ON DELETE CASCADE
        )
      ''')

      cursor.execute('''
        CREATE TABLE IF NOT EXISTS advisors_academic_titles (
          advisor_id INTEGER NOT NULL,
          academic_title_id INTEGER NOT NULL,
          PRIMARY KEY (advisor_id, academic_title_id),
          FOREIGN KEY (advisor_id) REFERENCES researchers (researcher_id) ON DELETE CASCADE,
          FOREIGN KEY (academic_title_id) REFERENCES academic_titles (academic_title_id) ON DELETE CASCADE
        )
      ''')

      # Inserindo os dados
      cursor.execute('''
        INSERT INTO researchers (researcher_id, name)
        VALUES (?, ?)
      ''', (self.researcher_id, self.name))

      for academic_title in self.academic_titles:
        cursor.execute('''
          INSERT INTO academic_titles (researcher_id, title, dissertation, institution, year, country)
          VALUES (?, ?, ?, ?, ?, ?)
        ''', (self.researcher_id, academic_title.title, academic_title.dissertation, academic_title.institution, academic_title.year, academic_title.country))
        academic_title_id = cursor.lastrowid
        if isinstance(academic_title.advisors_id, (list, tuple)):
          for advisor_id in academic_title.advisors_id:
            cursor.execute('''
              INSERT INTO advisors_academic_titles (advisor_id, academic_title_id)
              VALUES (?, ?)
            ''', (advisor_id, academic_title_id))

      conn.commit()

  def close_db(db_sqlite):
    conn = sqlite3.connect(db_sqlite)
    conn.close()

# Objeto título acadêmico
class academic_title:
  def __init__(self, title, dissertation, institution, year, country, advisors_id):
    self.title = title
    self.dissertation = dissertation
    self.institution = institution
    self.year = year
    self.country = country
    self.advisors_id = advisors_id

  def __str__(self):
    return f'{self.title} - {self.dissertation} - {self.institution} - {self.year} - {self.country} - {self.advisors_id}'

  def to_list(self):
    return [self.title, self.dissertation, self.institution, self.year, self.country, self.advisors_id]


class researcher_numeric_attributes:
  def __init__(self, researcher_dissertations, advisor_ids, student_ids):
    '''Retorna os atributos numéricos do pesquisador'''

    self.num_academic_titles = len(researcher_dissertations)
    self.first_year = min([diss[1] for diss in researcher_dissertations])
    self.last_year = max([diss[1] for diss in researcher_dissertations])
    self.num_advisors = len(advisor_ids)
    self.num_students = len(student_ids)
    self.average_years = average_between_years([diss[1] for diss in researcher_dissertations])

  def __str__(self):
    return f'{self.num_academic_titles} - {self.first_year} - {self.last_year} - {self.num_advisors} - {self.num_students} - {self.average_years}'

## Funções:

In [ ]:
def search_db(db_sqlite, query, params=()):
  '''Essa função usa query e parâmetros para buscar no banco'''

  with sqlite3.connect(db_sqlite) as conn:
        cursor = conn.cursor()
        cursor.execute(query, params)

        return cursor.fetchall()


def generate_sql_query(user_query):
    '''Essa função usa IA para converter pergunta em SQL parametrizado.'''

    system_prompt = """Você é um assistente especializado em bancos de dados SQLite. Converta perguntas em SQL parametrizado usando `?` para os valores.

    Estrutura do banco que você é especialista:
    researchers (
      researcher_id INTEGER PRIMARY KEY,
      name TEXT NOT NULL
    )

    academic_titles (
      academic_title_id INTEGER PRIMARY KEY,
      researcher_id INTEGER NOT NULL,
      title TEXT NOT NULL,
      dissertation TEXT,
      institution TEXT,
      year INTEGER,
      country TEXT,
      FOREIGN KEY (researcher_id) REFERENCES researchers (researcher_id) ON DELETE CASCADE
    )

    advisors_academic_titles (
      advisor_id INTEGER NOT NULL,
      academic_title_id INTEGER NOT NULL,
      PRIMARY KEY (advisor_id, academic_title_id),
      FOREIGN KEY (advisor_id) REFERENCES researchers (researcher_id) ON DELETE CASCADE,
      FOREIGN KEY (academic_title_id) REFERENCES academic_titles (academic_title_id) ON DELETE CASCADE
    )

    Exemplo de saída correta:
    ['SELECT name FROM researchers WHERE researcher_id = ?', ['9', '22']]

    Retorne apenas a SQL e os valores em uma lista Python, sem explicações. Não inclua comandos DELETE ou UPDATE.
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_query}
        ]
    )

    return eval(response.choices[0].message.content.strip())


def get_advisor_and_student_ids(researcher_id):
  '''Essa função retorna a lista de orientadores e alunos de um pesquisador'''

  # Busca todos os orientadores (advisor_id) que estão associados a títulos acadêmicos pertencentes a um pesquisador específico (researcher_id).
  advisor_ids = search_db(db_sqlite,
                          "SELECT aat.advisor_id\
                           FROM advisors_academic_titles aat\
                           JOIN academic_titles at ON aat.academic_title_id = at.academic_title_id\
                           WHERE at.researcher_id = ?", (researcher_id,))

  # Busca os IDs dos pesquisadores que têm pelo menos um título acadêmico orientado pelo pesquisador especificado (researcher_id)
  student_ids = search_db(db_sqlite,
                          "SELECT r.researcher_id\
                           FROM researchers r\
                           JOIN academic_titles at ON r.researcher_id = at.researcher_id\
                           JOIN advisors_academic_titles aat ON at.academic_title_id = aat.academic_title_id\
                           WHERE aat.advisor_id = ?", (researcher_id,))

  # Corrige a lista
  advisor_ids = [row[0] for row in advisor_ids]
  student_ids = [row[0] for row in student_ids]

  return advisor_ids, student_ids

def list_to_id_and_name(list_id):
  '''Essa função retorna uma lista com ID e nome de uma lista de ID'''

  list_id_and_name = search_db(db_sqlite,
                                "SELECT researcher_id, name\
                                 FROM researchers\
                                 WHERE researcher_id IN ({})".format(",".join("?" * len(list_id))),
                                tuple(list_id))

  return list_id_and_name


def get_dissertations_researcher_son(db_sqlite, researcher_id):
  '''Retorna dissertações do pesquisador e de seus alunos'''
  with sqlite3.connect(db_sqlite) as conn:
      cursor = conn.cursor()

      # Dissertações e ano do próprio pesquisador
      cursor.execute('''
          SELECT dissertation, year FROM academic_titles
          WHERE researcher_id = ?
      ''', (researcher_id,))
      researcher_dissertations = [row for row in cursor.fetchall()]

      # Dissertações e ano da dissertações dos alunos (alunos desse pesquisador são os que têm ele como orientador)
      cursor.execute('''
          SELECT at.dissertation, year FROM academic_titles at
          JOIN advisors_academic_titles aat ON at.academic_title_id = aat.academic_title_id
          WHERE aat.advisor_id = ?
      ''', (researcher_id,))
      students_dissertations = [row for row in cursor.fetchall()]

  return researcher_dissertations, students_dissertations


def average_between_years(list_year):
  if len(list_year) > 1:
    return np.median([list_year[i] - list_year[i - 1] for i in range(1, len(list_year))])
  else:
    return 0

## Consultas:

In [ ]:
# Busca toda a base de dados

df_all = pd.read_sql_query('''SELECT
                        r.researcher_id,
                        r.name AS researcher_name,
                        at.academic_title_id,
                        at.title AS academic_title,
                        at.dissertation,
                        at.institution,
                        at.year,
                        at.country,
                        GROUP_CONCAT(aat.advisor_id) AS advisor_ids  -- Lista os orientadores em uma única coluna
                    FROM academic_titles at
                    JOIN researchers r ON r.researcher_id = at.researcher_id
                    LEFT JOIN advisors_academic_titles aat ON at.academic_title_id = aat.academic_title_id
                    GROUP BY at.academic_title_id''', sqlite3.connect(db_sqlite))
sqlite3.connect(db_sqlite).close()

df_all.tail(10)

In [ ]:
# Busca orientadores e alunos de um pesquisador
researcher_id = 9#16

print(f'🔎 Buscando informações do pesquisador {researcher_id}...\n')
# Busca ID de todos os orientadores e alunos de um pesquisador
advisor_ids, student_ids = get_advisor_and_student_ids(researcher_id)


if advisor_ids:
    print(f'🎓 IDs dos orientadores encontrados: {advisor_ids}')

    # Busca o ID e o nome dos pesquisadores cujos researcher_id estão na lista de advisor_ids
    advisors = list_to_id_and_name(advisor_ids)
    print("📋 Lista de orientadores:")
    for advisor_id, advisor_name in advisors:
        print(f'🔹 ID: {advisor_id}, Nome: {advisor_name}')
else:
    print("❌ Nenhum orientador encontrado.")


print("\n" + "="*50 + "\n")   # Passa uma linha no terminal


if student_ids:
    print(f'🎓 IDs dos alunos encontrados: {student_ids}')

    # Busca os nomes e IDs dos pesquisadores cujos IDs estão na lista student_ids.
    students = list_to_id_and_name(student_ids)
    print("📋 Lista de alunos:")
    for student_id, student_name in students:
        print(f'🔹 ID: {student_id}, Nome: {student_name}')
else:
    print("❌ Nenhum aluno encontrado.")

In [ ]:
# Busca dissertações do pesquisador e de seus alunos
researcher_dissertations, students_dissertations = get_dissertations_researcher_son(db_sqlite, researcher_id)

print("Dissertações do pesquisador:",[diss[0] for diss in researcher_dissertations])
print("Dissertações dos alunos:", [diss[0] for diss in students_dissertations])

In [ ]:
print('Atributos númericos\n')
researcher_attributes = researcher_numeric_attributes(researcher_dissertations, advisor_ids, student_ids)

print(
      'Número de títulos acadêmicos:', researcher_attributes.num_academic_titles,
      '\nAno do primeiro e do último título:', researcher_attributes.first_year, 'e', researcher_attributes.last_year,
      '\nNúmero de orientadores:', researcher_attributes.num_advisors,
      '\nNúmero de alunos orientados:', researcher_attributes.num_students,
      '\nTempo médio entre seus títulos:', researcher_attributes.average_years,
      '\n\n',
      '\bTotal de pesquisadores:', search_db(db_sqlite, 'SELECT COUNT(*) FROM researchers')[0][0],
      '\nTotal de orientadores:', search_db(db_sqlite, 'SELECT COUNT(DISTINCT advisor_id) FROM advisors_academic_titles')[0][0],
      '\nTotal de não orientadores:', search_db(db_sqlite, 'SELECT COUNT(*) FROM researchers WHERE researcher_id NOT IN (SELECT advisor_id FROM advisors_academic_titles)')[0][0],
      )

# Utilizando inteligência artificial:

In [ ]:
# Pedir a chave sem exibi-la
#os.environ["OPENAI_API_KEY"] = getpass("Digite sua chave de API: ")

In [ ]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def chatbot(user_query):
    sql_data = generate_sql_query(user_query)
    print("🔍 Query gerada:", sql_data)

    try:
        results = search_db(db_sqlite, sql_data[0], sql_data[1])
        if results:
            return results
        return "Nenhum resultado encontrado."
    except Exception as e:
        return f"Erro ao executar a query: {e}"

# Testando
user_input = "Qual o total são orientadores"
print(chatbot(user_input))

# PyOperon

In [60]:
df_optimized = pd.read_feather("./data/mgp_optimized.feather")

df_optimized[256:261]

,researcher_id,researcher_name,academic_title_id,academic_title,dissertation,institution,year,country,advisor_ids,num_advisors,num_students,num_siblings,generation
256,258,Dio Lewis Holl,257,Ph.D.,Viscous Fluid Motion in Eccentric Cylinders,The University of Chicago,1925.0,UnitedStates,[38464],1,12,0,0
257,259,Roger Henry Homer,258,Ph.D.,Abstract Extension Theory of Operators in Bana...,"University of California, Berkeley",1959.0,UnitedStates,[32846],1,3,0,0
258,260,Dean L. Isaacson,259,Ph.D.,None,University of Minnesota - Twin Cities,1968.0,UnitedStates,[],0,2,0,0
259,261,Elgin Harold Johnston,260,Ph.D.,Growth of Derivatives of Approximations to Ana...,University of Illinois at Urbana-Champaign,1977.0,UnitedStates,[4603],1,3,0,0
260,262,Fritz Keinert,261,Ph.D.,The Divergent K-Plane Transform,Oregon State University,1985.0,UnitedStates,[8230],1,5,0,0


## Existe uma relação entre o número de alunos orientados por um pesquisador e características como instituição, país, ano do título e número de orientadores?

In [ ]:
df_optimized = pd.read_feather("./data/mgp_optimized.feather")

# Selecionar as colunas relevantes para prever o número de alunos
features = ["institution", "year", "country", "num_advisors", "num_siblings"]
target = "num_students"

#df_optimized = df_optimized.head(300)

# Remover valores nulos
print("Tamanho do banco com valores nulos:", len(df_optimized))
df_optimized = df_optimized.dropna(subset=features)
print("Novo tamanho do banco sem valores nulos:", len(df_optimized))

In [ ]:
### Codificar variáveis categóricas (institution, country)
label_encoders = {}
for col in ["institution", "country"]:
    le = LabelEncoder()
    df_optimized[col] = le.fit_transform(df_optimized[col].str.strip().str.lower().str.replace(" ", ""))
    label_encoders[col] = le

### Normalizar valores numéricos
#scaler = MinMaxScaler()
#df_optimized[features] = scaler.fit_transform(df_optimized[features])

### Converter em Fortran-order
#df_optimized["country_boost"] = df_optimized["country"] * 10
#features = ["institution", "year", "country_boost", "num_advisors"]
features_df = np.asfortranarray(df_optimized[features].values, dtype=np.float64)
target_df = np.asfortranarray(df_optimized[target].values, dtype=np.float64)

# Criar o regressor simbólico
model = SymbolicRegressor(
    n_threads=32
)

### Treinar o modelo com os dados
#model.fit(df_optimized[features], df_optimized[target])
model.fit(features_df, target_df)

## Exibir o melhor modelo encontrado
print("Melhor expressão encontrado:\n", model.pareto_front_[0]['model'], "\n")

In [ ]:
### Salvar os parâmetros do modelo
modelo_treinado = '/content/drive/My Drive/Doutorado/BraSNAM/modelo_treinado.pkl'

#pickle.dump(model, open(modelo_treinado, 'wb'))

In [ ]:
### Fazer previsões
df_optimized["predicted_students"] = model.predict(np.asfortranarray(df_optimized[features].values, dtype=np.float64))

### Visualizar algumas previsões
print(df_optimized[[target, "predicted_students"]].head(20))

### Calcular o erro absoluto
mae = mean_absolute_error(df_optimized[target], df_optimized["predicted_students"])
print("\nErro médio absoluto:", mae)

In [ ]:
print("Predição com mais alunos:\n\n", df_optimized[features + ["predicted_students"]].nlargest(10, "predicted_students"))


In [ ]:
# Comparar informações de dois IDs
id_find1 = 4
id_find2 = 1
print(
    df_optimized.iloc[id_find1]["institution"],
    df_optimized.iloc[id_find1]["year"],
    df_optimized.iloc[id_find1]["country"],
    df_optimized.iloc[id_find1]["num_advisors"]
    )
print(
    df_optimized.iloc[id_find2]["institution"],
    df_optimized.iloc[id_find2]["year"],
    df_optimized.iloc[id_find2]["country"],
    df_optimized.iloc[id_find2]["num_advisors"]
    )

In [ ]:
# Perguntar sobre um novo pesquisador

#input_institution = input("Instituição: ")
#input_year = int(input("Ano: "))
#input_country = input("País: ")
#input_num_advisors = int(input("Número de orientadores: "))

researcher_new = {
    "institution": "Iowa State University",
    "year": 2024,
    "country": "United States",
    "num_advisors": 7
    }


# Aplicando a transformação aos inputs categóricos
for col in ["institution", "country"]:
    researcher_new[col] = researcher_new[col].strip().lower().replace(" ", "")
    if researcher_new[col] in label_encoders[col].classes_:
        researcher_new[col] = int(label_encoders[col].transform([researcher_new[col]])[0])
    else:
        print(f"Aviso: '{researcher_new[col]}' não está nos dados de treinamento. Usando valor padrão -1.")
        researcher_new[col] = -1  # Valor para desconhecidos

print("Dados de entrada:", researcher_new)
researcher_fortran = np.asfortranarray([[researcher_new[col] for col in features]], dtype=np.float64)

print("Pesquisador exemplo deve ter:", max(1, round(model.predict(researcher_fortran)[0])), "aluno(s)")

## Qual é o grau de similaridade temática entre a dissertação de um pesquisador, as de seus orientadores e de seus alunos?

In [ ]:
# Carregar modelo BERT
bert_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Criar embeddings dos orientadores
print("\n\nTamanho do banco com valores nulos:", len(df_optimized))
df_sem_nulos = df_optimized["dissertation"].dropna()
print("Novo tamanho do banco sem valores nulos:", len(df_sem_nulos))
orientadores_emb = {idx: bert_model.encode(text) for idx, text in tqdm(df_sem_nulos.items(), desc="Gerando Embeddings")}

# Criar DataFrame de treinamento
dados = []
for nome, emb in orientadores_emb.items():
    features = emb  # Embedding BERT como features
    dados.append([nome, *features])

df_train = pd.DataFrame(dados, columns=["orientador"] + [f"feat_{i}" for i in range(len(features))])

# Criar similaridade como alvo (usamos o próprio cosseno como target para ensinar o modelo)
# Calcular similaridade apenas para pares necessários (diagonal)
similarities = []
embeddings = list(orientadores_emb.values())
for emb in tqdm(embeddings, desc="Calculando similaridades"):
    similarity = cosine_similarity([emb], [emb])[0, 0]  # Similaridade com ela mesma
    similarities.append(similarity)

df_train["similarity"] = similarities  # Similares com elas mesmas (idealmente, dados rotulados viriam de alunos reais)

# Treinar modelo de Regressão Simbólica
X = df_train.drop(columns=["orientador", "similarity"]).values  # Features (Embeddings)
y = df_train["similarity"].values  # Similaridade como target

model = SymbolicRegressor()

print("\n\nTreinamento iniciado...\n")
model.fit(X, y)
print("...treinamento finalizado.")

# Função para encontrar o melhor orientador para uma nova dissertação
def melhor_orientador(nova_dissertacao):
    nova_emb = bert_model.encode(nova_dissertacao).reshape(1, -1)  # Gerar embedding
    pred_sim = model.predict(nova_emb)[0]  # Calcular similaridade

    # Comparar com todos os orientadores
    melhor_nome = max(orientadores_emb.keys(), key=lambda nome: cosine_similarity([orientadores_emb[nome]], nova_emb)[0, 0])

    return melhor_nome, pred_sim

In [ ]:
#input_dissertacao = input("Digite a dissertação: ")
# Testando com uma nova dissertação
nova_dissertacao = "Using symbolic regression in predictions"
#nova_dissertacao = "Statical Equilibrium of Skew"
melhor, score = melhor_orientador(nova_dissertacao)

melhor_researcher = df_optimized.iloc[melhor]
#print(f"Melhor orientador para essa dissertação: {melhor_researcher} (similaridade: {score:.3f})")
print(f"🚀 Melhor orientador para essa dissertação 🚀\nID MGP: {melhor_researcher['researcher_id']}\nNome: {melhor_researcher['researcher_name']}\nDissertação: {melhor_researcher['dissertation']} (similaridade: {score:.2%})")

In [ ]:
#FIM